In [1]:
import os

In [2]:
%pwd

'e:\\Deep-learning-credit-card-fraud-detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Deep-learning-credit-card-fraud-detection'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Model_trainerConfig:
    root_dir :Path
    model_dir :Path
    train_dir :Path
    test_dir : Path
    processor_dir :Path


In [6]:
from src.card_fraud_detection.utilis.commmon import create_directories,save_model
from src.card_fraud_detection.utilis.commmon import read_yaml
from src.card_fraud_detection.constants import config_file_path,params_file_path

In [8]:
class Confuguration:
    def __init__(self,confi_path=config_file_path,params_path=params_file_path):
        self.config=read_yaml(confi_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.Artifacts])

    def get_model_trainer_config(self)->Model_trainerConfig:
        trainer=self.config.Model_trainer
        ingestion =self.config.Data_ingestion
        transformation=self.config.data_transformation

        create_directories([trainer.root_dir])

        trainer=Model_trainerConfig(
            root_dir=trainer.root_dir,
            model_dir=trainer.model_dir,
            train_dir=ingestion.train_dir,
            test_dir=ingestion.test_dir,
            processor_dir= transformation.processor_dir
        )
        return trainer


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
import tensorflow as tf
import pandas as pd

In [21]:
class Model_trainer:
    def __init__(self,config):
        self.config=config

    def get_data_trainer(self):

        x_train=pd.read_csv(self.config.train_dr)


        model=Sequential()
        model.add(Dense(64,activation='relu',input_dim=x_train.shape[1]))
        model.add(Dense(64,activation='relu'))
        model.add(Dense(1,activation='sigmoid'))

        model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=tf.keras.metrics.Recall(
        thresholds=None, top_k=None, class_id=None, name=None, dtype=None))
        return model

    def model_trainer_initiated(self,x_train,y_train,x_test,y_test):
        model=self.get_data_trainer()
        histroy=model.fit(x_train,y_train,batch_size=120,epochs=30)

        save_model(filepath=self.config.model_dir,obj=model)
        return histroy

In [18]:
from src.card_fraud_detection.component.data_transformation import DataTransformation

In [20]:
try:
    trainer_config=Confuguration()
    config=trainer_config.get_model_trainer_config()

    data_trans=DataTransformation(config=config)
    x_train,y_train,x_test,y_test=data_trans.data_transformation_initiated()



    model_trainer=Model_trainer(config=config)

    histroy=model_trainer.model_trainer_initiated(x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)

    print(histroy)

except Exception as e:
    raise e

[2023-08-30 15:43:49,926: INFO: commmon: file created]
[2023-08-30 15:43:49,928: INFO: commmon: file created]


(235670, 163)
(235670, 1)
Epoch 1/30
1964/1964 [==============================] - 10s 4ms/step - loss: 0.0101 - recall_2: 0.9585
Epoch 2/30
1964/1964 [==============================] - 7s 4ms/step - loss: 3.0789e-04 - recall_2: 0.9968
Epoch 3/30
1964/1964 [==============================] - 7s 4ms/step - loss: 2.1124e-04 - recall_2: 0.9977
Epoch 4/30
1964/1964 [==============================] - 7s 3ms/step - loss: 2.1700e-04 - recall_2: 0.9972
Epoch 5/30
1964/1964 [==============================] - 7s 4ms/step - loss: 2.1903e-04 - recall_2: 0.9979
Epoch 6/30
1964/1964 [==============================] - 7s 3ms/step - loss: 1.7613e-04 - recall_2: 0.9979
Epoch 7/30
1964/1964 [==============================] - 7s 4ms/step - loss: 1.7690e-04 - recall_2: 0.9982
Epoch 8/30
1964/1964 [==============================] - 9s 4ms/step - loss: 1.6811e-04 - recall_2: 0.9979
Epoch 9/30
1964/1964 [==============================] - 8s 4ms/step - loss: 1.8995e-04 - recall_2: 0.9979
Epoch 10/30
1964/1964 [